In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

from plotly.subplots import make_subplots

In [2]:
import seaborn as sns
# Apply seaborn style
sns.set()

mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [3]:
df = pd.read_csv('2024-07-05_BTC-USD_1m.csv', index_col=0, parse_dates=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7239 entries, 2024-06-28 00:01:00+00:00 to 2024-07-04 23:58:00+00:00
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       7239 non-null   float64
 1   High       7239 non-null   float64
 2   Low        7239 non-null   float64
 3   Close      7239 non-null   float64
 4   Adj Close  7239 non-null   float64
 5   Volume     7239 non-null   int64  
 6   Symbol     7239 non-null   object 
dtypes: float64(5), int64(1), object(1)
memory usage: 452.4+ KB


In [4]:
# Check for missing values
print("Missing values before dropping:")
print(df.isna().sum())

Missing values before dropping:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
Symbol       0
dtype: int64


In [5]:
SMA1 = 42
SMA2 = 252

# Calculate moving averages
df['SMA1'] = df['Close'].rolling(SMA1).mean()
df['SMA2'] = df['Close'].rolling(SMA2).mean()

In [6]:
import plotly.graph_objects as go

# Function to plot the graph
def plot_graph(df):
    fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])

    # Plot close price
    fig.add_trace(
        go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price', line=dict(color="blue", width=1)),
        row=1, col=1
    )

    # Plot SMA1
    fig.add_trace(
        go.Scatter(x=df.index, y=df['SMA1'], mode='lines', name=f'SMA{SMA1}', line=dict(color="orange", width=1)),
        row=1, col=1
    )

    # Plot SMA2
    fig.add_trace(
        go.Scatter(x=df.index, y=df['SMA2'], mode='lines', name=f'SMA{SMA2}', line=dict(color="green", width=1)),
        row=1, col=1
    )

    # Plot Position on secondary y-axis
    fig.add_trace(
        go.Scatter(x=df.index, y=df['Position'], mode='lines', name='Position', line=dict(color="red", width=1)),
        secondary_y=True, row=1, col=1
    )

    fig.update_layout(
        title={'text':'Price and Moving Averages', 'x':0.5},
        autosize=False,
        width=800, height=400
    )

    # Set y-axis range for the main plot
    fig.update_yaxes(title_text="Price", range=[df['Close'].min(), df['Close'].max()], secondary_y=False)

    # Set y-axis title for the secondary plot
    fig.update_yaxes(title_text="Position", secondary_y=True)

    # Hide the secondary y-axis range slider
    fig.update_yaxes(visible=False, secondary_y=True)
    
    fig.show()

In [7]:
df['Position'] = np.where(df['SMA1'] > df['SMA2'], 1, -1) 

In [8]:
# Plot the graph
plot_graph(df)

Log Return
Log Return, also known as the natural logarithm of the return, is a way to measure the relative change in the price of an asset. It is calculated using the formula:

Log Return = ln(Pt /Pt-1)

Why Use Log Returns?
1. Additivity: Log returns are additive over time. This means that the log return over multiple periods can be obtained by summing the log returns of the individual periods. For example, the log return over a week is the sum of the log returns for each day of the week.
2. Symmetry: Log returns treat upward and downward movements symmetrically. For example, an increase of 10% and a decrease of 10% will have the same magnitude in log returns, but with opposite signs.


In [9]:
# Calculate Log Returns
df['Returns'] = np.log(df['Close'] / df['Close'].shift(1))

# Calculate Strategy Returns - Avoid foresight bias
df['Strategy'] = df['Position'].shift(1) * df['Returns']

df.round(4).head()

,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA1,SMA2,Position,Returns,Strategy
Datetime,,,,,,,,,,,,
2024-06-28 00:01:00+00:00,61613.2070,61613.2070,61613.2070,61613.2070,61613.2070,0,BTC-USD,NaN,NaN,-1,NaN,NaN
2024-06-28 00:02:00+00:00,61575.1406,61575.1406,61575.1406,61575.1406,61575.1406,0,BTC-USD,NaN,NaN,-1,-0.0006,0.0006
2024-06-28 00:04:00+00:00,61596.7305,61596.7305,61596.7305,61596.7305,61596.7305,1658880,BTC-USD,NaN,NaN,-1,0.0004,-0.0004
2024-06-28 00:05:00+00:00,61577.5938,61577.5938,61577.5938,61577.5938,61577.5938,0,BTC-USD,NaN,NaN,-1,-0.0003,0.0003
2024-06-28 00:06:00+00:00,61575.2070,61575.2070,61575.2070,61575.2070,61575.2070,0,BTC-USD,NaN,NaN,-1,-0.0000,0.0000


In [10]:
df.dropna(inplace=True) # Drop missing values
df.round(4).head()

,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA1,SMA2,Position,Returns,Strategy
Datetime,,,,,,,,,,,,
2024-06-28 05:16:00+00:00,61571.5781,61571.5781,61571.5781,61571.5781,61571.5781,0,BTC-USD,61569.1061,61699.9508,-1,0.0001,-0.0001
2024-06-28 05:17:00+00:00,61558.0352,61558.0352,61558.0352,61558.0352,61558.0352,0,BTC-USD,61568.9244,61699.7318,-1,-0.0002,0.0002
2024-06-28 05:18:00+00:00,61555.4766,61555.4766,61555.4766,61555.4766,61555.4766,0,BTC-USD,61569.3865,61699.6538,-1,-0.0000,0.0000
2024-06-28 05:19:00+00:00,61547.5664,61547.5664,61547.5664,61547.5664,61547.5664,0,BTC-USD,61570.9367,61699.4587,-1,-0.0001,0.0001
2024-06-28 05:20:00+00:00,61518.9922,61518.9922,61518.9922,61518.9922,61518.9922,0,BTC-USD,61572.0357,61699.2262,-1,-0.0005,0.0005


In [11]:
np.exp(df[['Returns', 'Strategy']].sum()) # Absolute performance

Returns     0.927278
Strategy    1.103109
dtype: float64

In [12]:
# The factor 252 ** 0.5 is used to annualize the standard deviation, 
# assuming there are 252 trading days in a year.
df[['Returns', 'Strategy']].std() * 252 ** 0.5 # Annualized volatility (standard deviation)

Returns     0.009930
Strategy    0.009929
dtype: float64

In [13]:
sma1_range = range(20, 61, 4)
sma2_range = range(180, 281, 10)

In [14]:
from itertools import product

# Initialize an empty list to store results
results_list = []

# Iterate over the product of sma1_range and sma2_range
for SMA1, SMA2 in product(sma1_range, sma2_range):
    data = df.copy()
    
    # Calculate log returns
    data['Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    
    # Calculate moving averages
    data[f'SMA{SMA1}'] = data['Close'].rolling(SMA1).mean()
    data[f'SMA{SMA2}'] = data['Close'].rolling(SMA2).mean()
    
    # Drop NaNs introduced by rolling window
    data.dropna(inplace=True)
    
    # Generate trading positions: 1 if SMA1 > SMA2 else -1
    data['Position'] = np.where(data[f'SMA{SMA1}'] > data[f'SMA{SMA2}'], 1, -1)
    
    # Shift positions by 1 to align with returns
    data['Strategy'] = data['Position'].shift(1) * data['Returns']
    
    # Drop any NaNs introduced by the shift operation
    data.dropna(inplace=True)
    
    # Calculate cumulative performance
    perf = np.exp(data[['Returns', 'Strategy']].sum())
    
    # Append the performance results to the results list
    results_list.append({
        'SMA1': SMA1,
        'SMA2': SMA2,
        'MARKET': perf['Returns'],
        'STRATEGY': perf['Strategy'],
        'OUT': perf['Strategy'] - perf['Returns']
    })

In [15]:
results = pd.DataFrame(results_list) # Create a DataFrame from the results list
results.head()

,SMA1,SMA2,MARKET,STRATEGY,OUT
0,20,180,0.930958,1.078963,0.148005
1,20,190,0.930054,1.088170,0.158117
2,20,200,0.929798,1.092471,0.162674
3,20,210,0.930401,1.088571,0.158170
4,20,220,0.930451,1.090084,0.159633


In [16]:
results.info() # Check the data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SMA1      121 non-null    int64  
 1   SMA2      121 non-null    int64  
 2   MARKET    121 non-null    float64
 3   STRATEGY  121 non-null    float64
 4   OUT       121 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 4.9 KB


In [17]:
# Sort the results by 'OUT' in descending order and display the top 7 rows
top_results = results.sort_values('OUT', ascending=False).head(7)
print(top_results)

# Display the optimal parameters
optimal_params = top_results.iloc[0]
print("\nOptimal Parameters:")
print(optimal_params)

    SMA1  SMA2    MARKET  STRATEGY       OUT
58    40   210  0.930401  1.133116  0.202714
59    40   220  0.930451  1.128649  0.198198
47    36   210  0.930401  1.127958  0.197557
46    36   200  0.929798  1.126119  0.196321
56    40   190  0.930054  1.121026  0.190972
48    36   220  0.930451  1.120901  0.190450
38    32   230  0.929174  1.118727  0.189553

Optimal Parameters:
SMA1         40.000000
SMA2        210.000000
MARKET        0.930401
STRATEGY      1.133116
OUT           0.202714
Name: 58, dtype: float64


In [18]:
data = df.copy()

lags = 5  # Number of previous days to use as features
cols = []

for lag in range(1, lags + 1):
    col = 'lag_{}'.format(lag)  
    data[col] = data['Close'].shift(lag)

In [19]:
data.head(7)

,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA1,SMA2,Position,Returns,Strategy,lag_1,lag_2,lag_3,lag_4,lag_5
Datetime,,,,,,,,,,,,,,,,,
2024-06-28 05:16:00+00:00,61571.578125,61571.578125,61571.578125,61571.578125,61571.578125,0,BTC-USD,61569.106120,61699.950784,-1,0.000122,-0.000122,NaN,NaN,NaN,NaN,NaN
2024-06-28 05:17:00+00:00,61558.035156,61558.035156,61558.035156,61558.035156,61558.035156,0,BTC-USD,61568.924386,61699.731848,-1,-0.000220,0.000220,61571.578125,NaN,NaN,NaN,NaN
2024-06-28 05:18:00+00:00,61555.476562,61555.476562,61555.476562,61555.476562,61555.476562,0,BTC-USD,61569.386533,61699.653816,-1,-0.000042,0.000042,61558.035156,61571.578125,NaN,NaN,NaN
2024-06-28 05:19:00+00:00,61547.566406,61547.566406,61547.566406,61547.566406,61547.566406,0,BTC-USD,61570.936663,61699.458721,-1,-0.000129,0.000129,61555.476562,61558.035156,61571.578125,NaN,NaN
2024-06-28 05:20:00+00:00,61518.992188,61518.992188,61518.992188,61518.992188,61518.992188,0,BTC-USD,61572.035714,61699.226175,-1,-0.000464,0.000464,61547.566406,61555.476562,61558.035156,61571.578125,NaN
2024-06-28 05:21:00+00:00,61496.132812,61496.132812,61496.132812,61496.132812,61496.132812,0,BTC-USD,61572.473772,61698.912388,-1,-0.000372,0.000372,61518.992188,61547.566406,61555.476562,61558.035156,61571.578125
2024-06-28 05:22:00+00:00,61503.761719,61503.761719,61503.761719,61503.761719,61503.761719,4829184,BTC-USD,61572.607794,61698.423007,-1,0.000124,-0.000124,61496.132812,61518.992188,61547.566406,61555.476562,61558.035156


In [20]:
data.dropna(inplace=True)